<div style="background-image: linear-gradient(to right, #4b4cff , #00d4ff); text-align: center; padding: 50px;">
    <h1 style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; font-size: 24px; color: white; text-shadow: 2px 2px #4b4cff;">
        Json to Dashboard
    </h1>
</div>

<h2 style="color: #3366cc; font-family: Arial, sans-serif; font-size: 24px; font-weight: bold; text-transform: uppercase; text-align: center; border-bottom: 2px solid #3366cc; padding-bottom: 5px;">Bibliothèque</h2>

In [99]:
import json

from bokeh.plotting import figure, show
from bokeh.io import curdoc, output_file, save
from bokeh.models import ColumnDataSource, FactorRange, Div, CategoricalTicker, TableColumn
from bokeh.layouts import column , layout, gridplot

import os

<h2 style="color: #3366cc; font-family: Arial, sans-serif; font-size: 24px; font-weight: bold; text-transform: uppercase; text-align: center; border-bottom: 2px solid #3366cc; padding-bottom: 5px;">Graph</h2>

In [100]:
def binary_graph(data) : 
    """
    Prépare les graphiques de type binaires

    Arguments :
    - data : les données en format json. 

    Retourne :
    - plots : Les graphes en barre binaires préparés.
    - title : Le titre de la catégorie.
    """
    binary_columns = []

    for column, var in data['variables'].items():
        if var['type'] == 'binary':
            binary_columns.append(column)
    # Récupération des colonnes de type 'binary'
    
    plots = []
    # Initialisation des plots
    
    for col in binary_columns:
        
        values = [data['variables'][col]['appearance']]
        binary_keys = list(values[0].keys())
        binary_values = list(values[0].values())
        # Récupération des labels et des values de chaque colonne

        x_range = binary_keys
        source = ColumnDataSource(data=dict(x=x_range, value=binary_values))
        # Mise en place des données

        bin_graph = figure(x_range=x_range, plot_height=400, plot_width=400, title=f"{col}")
        bin_graph.title.align = "center"
        # Initialisation de la figure
        
        bin_graph.vbar(x='x', top='value', width=0.4, legend_label='Valeur', source=source, color='steelblue', alpha=0.8)
        bin_graph.xgrid.grid_line_color = None
        # Style
        
        bin_graph.y_range.start = 0
        bin_graph.xaxis.axis_label = "Valeurs binaires"
        bin_graph.yaxis.axis_label = "Pourcentage"
        # Axes

        bin_graph.legend.label_text_font_size = '10pt'
        bin_graph.legend.location = "top_right"
        # Legend

        bin_graph.title.text_font_size = "14pt"
        # Style du titre
        
        plots.append(bin_graph)

    title = Div(text="<h1 style='text-align: center; color: #31EC56; font-size: 24px; font-weight: bold;'>GRAPHIQUE BINAIRE</h1>")
    # Titre de la section 
    
    return plots , title 

In [101]:
def id_graph(data):
    """
    Prépare les graphiques de type identifiant

    Arguments :
    - data : les données en format json. 

    Retourne :
    - plots : Les graphes en barre binaires préparés.
    - title : Le titre de la catégorie.
    """
    id_columns = []

    for column, var in data['variables'].items():
        if var['type'] == 'Identifiant':
            id_columns.append(column)
    # Récupération des colonnes de type 'Identifiants'
    
    plots = []

    for col in id_columns:
        values = [data['variables'][col]['>10% appearance']]
        id_keys = list(values[0].keys())
        id_values = list(values[0].values())
        # Récupération des labels et des values de chaque colonne
        
        x_range = id_keys
        source = ColumnDataSource(data=dict(x=x_range, value=id_values))
        # Mise en place des données

        id_graph = figure(x_range=x_range, plot_height=400, plot_width=400, title=f"{col}")
        id_graph.title.align = "center"
        # Initialisation de la figure

        id_graph.vbar(x='x', top='value', width=0.4, legend_label='Valeur', source=source, color='steelblue', alpha=0.8)
        id_graph.xgrid.grid_line_color = None
        # Style du graphe
        
        id_graph.y_range.start = 0
        id_graph.xaxis.axis_label = "Valeurs binaires"
        id_graph.yaxis.axis_label = "Pourcentage"
        # Axes
        
        id_graph.legend.label_text_font_size = '10pt'
        id_graph.legend.location = "top_right"
        # Legend
        
        id_graph.title.text_font_size = "14pt"

        plots.append(id_graph)
        # Ajout des graphes 
        
    title = Div(text="<h1 style='text-align: center; color: #EF036C; font-size: 24px; font-weight: bold;'>GRAPHIQUE IDENTIFIANTS 10% </h1>")
    return plots, title

In [102]:
def tableau(data): 
    """
    Prépare les graphiques des données continus (float64/int64)

    Arguments :
    - data : les données en format json. 

    Retourne :
    - plots : Les graphes en barre binaires préparés.
    - title : Le titre de la catégorie.
    """
    float_cols = []    
    
    for column, var in data['variables'].items():
        if var['type'] == 'int64' or var['type'] == 'float64':
            float_cols.append(column)
    # Récupération des colonnes 
    
    
    divs = []
    
    for col in float_cols: 
        values = data['variables'][col]
        
        text = f"<b>Colonne :</b> {col}<br>"
        text += f"<b>Type :</b> {values['type']}<br>"
        
        if 'statistics' in values:
            statistics = values['statistics']
            text += f"<b>Valeurs manquantes :</b> {values['missing values']}<br>"
            text += f"<b>Pourcentage valeurs manquantes pour la colonne:</b> {values['missing values percent column']}%<br>"
            text += f"<b>Pourcentage total valeurs manquantes :</b> {values['missing values percent total']}%<br>"
            text += f"<b>Valeur maximale :</b> {statistics['max']}<br>"
            text += f"<b>Valeur minimale :</b> {statistics['min']}<br>"
            text += f"<b>Valeur moyenne :</b> {statistics['mean']}<br>"
            text += f"<b>Ecart type :</b> {statistics['std']}<br>"
            text += f"<b>Quartile :</b> q1={statistics['q1']}, q3={statistics['q3']}<br>"
            # Informations pour les colonnes possédant des statistiques
        
        if 'extremum' in values:
            extremum = values['extremum']
            text += f"<b>Valeur minimale :</b> {extremum['earliest_year']}<br>"
            text += f"<b>Valeur maximale :</b> {extremum['latest_year']}<br>"
            # Informations pour les colonnes numériques de la forme 'annee'
        
        div = Div(text=text, width=400)  
        divs.append(div)
        # Ajout des informations
        
    title = Div(text="<h1 style='text-align: center; color: black; font-size: 24px; font-weight: bold;'>Informations numériques</h1>")
    return divs, title

In [103]:
def date_tab(data) :
    """
    Prépare les graphiques des données de type date

    Arguments :
    - data : les données en format json. 

    Retourne :
    - plots : Les graphes en barre binaires préparés.
    - title : Le titre de la catégorie.
    """
    date_cols = []
    
    for column, var in data['variables'].items():
        if var['type'] == 'datetime64[ns]':
            date_cols.append(column)
    # Récupération des colonnes de type date
    
    divs = []
    
    for col in date_cols : 
        values = data['variables'][col]
        
        text = f"<b>Colonne :</b> {col}<br>"
        text += f"<b>Type :</b> {values['type']}<br>"
        text += f"<b>Valeurs manquantes :</b> {values['missing values']}<br>"
        text += f"<b>Format :</b> {values['date format']}<br>"
        
        extremum = values['extremum']
        text += f"<b>Valeur minimale :</b> {extremum['earliest_date']}<br>"
        text += f"<b>Valeur maximale :</b> {extremum['latest_date']}<br>"
        # Traitement des valeurs des colonnes de type date
        
        div = Div(text=text, width=400) 
        divs.append(div)
        # Ajout des informations
    
    title = Div(text="<h1 style='text-align: center; color: black; font-size: 24px; font-weight: bold;'>Date</h1>")
    return divs, title

In [104]:
def dashboard(data) : 
    """
    Renvoie un dashboard préparéavec les fonctions des graphes précédentes

    Arguments :
    - file : un chemin vers le fichier .json du document.   

    Retourne :
    - dashboard : le dashboard du fichier passé
    """
    plots_bin, title_bin = binary_graph(data)
    plots_id, title_id = id_graph(data)
    div, title_tab = tableau(data)
    div_date, title_date = date_tab(data)
    # Récupération des informations

    column1 = column([title_bin, *plots_bin])
    column2 = column([title_id, *plots_id])
    column3 = column([title_tab, *div])
    column4 = column([title_date, *div_date])
    # Mise en forme en colonne

    dashboard = layout([[column1, column2, column3, column4]], sizing_mode='scale_width')

    output_file("./dashboard/dashboard.html")
    show(dashboard)

In [105]:
with open('./data/fr-en-ips_colleges.csv.json', 'r') as file:
    data = json.load(file)
# Récupération du fichier .json

dashboard(data)